In [1]:
# reload code if library changes
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [2]:
import os
if os.path.basename(os.getcwd()) == "notebooks": os.chdir("..")
import numpy as np

In [3]:
import datajoint as dj
from workflow_calcium_imaging.pipeline import *
populate_settings = {'display_progress': True}

[2023-03-02 12:19:34,676][INFO]: Connecting jure@127.0.0.1:3306
[2023-03-02 12:19:34,771][INFO]: Connected jure@127.0.0.1:3306


In [4]:
# move to library
def get_metadata_from_filetree(root_data_dir, fake_session_datetime_str_init):
    all_subject_str = []
    all_session_str = [] # this will be list of lists - each nested list corresponding to one subject
    all_datetime_str = [] # for now hardcoded! (figure out how to do programmatically)

    count = 0
    for subject_str in os.listdir(root_data_dir):
        if os.path.isdir(f'{root_data_dir}/{subject_str}'):

            print(f'Subject: {subject_str}')
            all_subject_str.append(subject_str)

            all_subject_session_str = [] # sessions for this particular subject
            all_subject_datetime_str = []
            for subject_session_str in os.listdir(root_data_dir + '/' + subject_str):
                all_subject_session_str.append(subject_session_str)
                fake_session_datetime_str = fake_session_datetime_str_init[:18] + str(count) + '.000' # making fake unique time
                print('\n\n\nIMPORTANT: JM made up a fake datetime to fit convention of DJ. If needed for analysis, the true datetime of the experiment can still be accessed though through the `session` entry (YYYY-MM-DD_x) or from where the bruker metadata is stored within the database.\n\n\n')
                print(fake_session_datetime_str)
                all_subject_datetime_str.append(fake_session_datetime_str) # here it is fake
                count += 1

            print(f'Identified sessions for subject {subject_str}: {all_subject_session_str}')

            all_session_str.append(all_subject_session_str) 
            all_datetime_str.append(all_subject_datetime_str) 
        
    return all_subject_str, all_session_str, all_datetime_str

### Clear previous s2p entries

In [5]:
imaging.ProcessingParamSet.delete()
imaging.Curation.delete()

[2023-03-02 12:19:34,984][INFO]: Deleting 1 rows from `jure_imaging`.`processing_task`
[2023-03-02 12:19:34,992][INFO]: Deleting 1 rows from `jure_imaging`.`#processing_param_set`


KeyboardInterrupt: Interrupted by user

In [ ]:
# custom function to populate database
root_data_dir = dj.config['custom']['imaging_root_data_dir']
fake_session_datetime_str_init = '2002-01-01 12:00:00.000' # making up session datetime (to query data use either the session/folder name or PraireView metadata)
all_subject_str, all_session_str, all_datetime_str = get_metadata_from_filetree(root_data_dir, fake_session_datetime_str_init)


Automated Suite2p

In [ ]:
choose_params = 'ops_jm_30hz_1plane_2ch' # number of channels, imaging rate and number of planes is overwritten by datajoint (see code for imaging.Processing.populate)

params_suite2p = np.load(f'/home/cossart/deve-networks/scripts/s2p_ops/{choose_params}.npy', allow_pickle=True).item()

params_suite2p

In [ ]:

params_suite2p['look_one_level_down'] = 0
params_suite2p['input_format'] = 'tif'
params_suite2p['bruker'] = False
params_suite2p['force_sktiff'] = True
params_suite2p['two_step_registration'] = 0.0
params_suite2p['fast_disk'] = 'suite2p_fast_disk'
# params_suite2p['functional_chan'] = 2 # THIS CHANGES IN SINGLE VS MULTI-CHANNEL
# params_suite2p['align_by_chan'] = 1 # THIS CHANGES IN SINGLE VS MULTI-CHANNEL
params_suite2p['tau'] = 0.15
params_suite2p['batch_size'] = 5000


# these should be input automatically by datajoint
# params_suite2p.pop('nplanes')
# params_suite2p.pop('nchannels')
# params_suite2p.pop('fs')

params_suite2p

In [ ]:
imaging.ProcessingParamSet.insert_new_params(
    processing_method='suite2p', 
    paramset_idx=0, 
    params=params_suite2p,
    paramset_desc=f'Calcium imaging analysis with Suite2p using {choose_params} parameters')

In [ ]:
from workflow_calcium_imaging import process
# process.run()

## Running suite2p for one animal

In [ ]:
session.SessionDirectory()

In [ ]:
session_key = (session.SessionDirectory & 'subject="jm007"').fetch('KEY')[0]
print(session_key )
session_dir_key = (session.SessionDirectory & 'subject="jm007"').fetch('session_dir')[0]

imaging.ProcessingTask.insert1(dict(session_key, 
                                    scan_id=0,
                                    paramset_idx=0,
                                    processing_output_dir=f'{session_dir_key}', 
                                    task_mode='trigger'))

In [ ]:
session_dir_key

## Populate `imaging.Processing`

For 3 plane 2 channel recording 2k batch: 

8016 frames of binary, time 1164.63 sec.

For 1 plane 2 channel recording 5k batch:

In [ ]:
imaging.Processing.populate(**populate_settings);

In [ ]:
imaging.Processing()

## Insert new Curation following the ProcessingTask

+ The next step in the pipeline is the curation of motion corection and segmentation results.

+ If a manual curation was implemented, an entry needs to be manually inserted into the table `imaging.Curation`, which specifies the directory to the curated results in `curation_output_dir`. 

+ If we would like to use the processed outcome directly, an entry is also needed in `imaging.Curation`. A method `create1_from_processing_task` was provided to help this insertion. It copies the `processing_output_dir` in `imaging.ProcessingTask` to the field `curation_output_dir` in the table `imaging.Curation` with a new `curation_id`.

    + In this example, we create/insert one `imaging.Curation` for each `imaging.ProcessingTask`, specifying the same output directory.

    + To this end, we could also make use of a convenient function `imaging.Curation().create1_from_processing_task()`

In [ ]:
imaging.Curation.heading

In [ ]:
# temp just to try the 2 plane 2 channel:
import datetime
foo_dict = {'subject': 'jm007', 'session_datetime': datetime.datetime(2002, 1, 1, 12, 0, 8)}

imaging.Curation.insert1(dict(subject=foo_dict['subject'], 
                                session_datetime=foo_dict['session_datetime'], 
                                scan_id=0,
                                paramset_idx=0,
                                curation_id=0,
                                curation_time=foo_dict['session_datetime'], 
                                curation_output_dir='jm007/2022-06-14_a/TSeries-05182022-1006-001/suite2p',
                                manual_curation=False,
                                curation_note=''))

In [ ]:
for (i, subject_str) in enumerate(all_subject_str):
    #for (j, subject_session_str) in enumerate(all_session_str[i]): # ONLY RUNNING FOR FIRST SESSION
    
    j = 0
    subject_session_str = all_session_str[i][j]
    
    session_datetime_str = all_datetime_str[i][j]

    imaging.Curation.insert1(dict(subject=subject_str, 
                                    session_datetime=session_datetime_str, 
                                    scan_id=0,
                                    paramset_idx=0,
                                    curation_id=0,
                                    curation_time=session_datetime_str, 
                                    curation_output_dir=f'{subject_str}/{subject_session_str}/TSeries/suite2p',
                                    manual_curation=False,
                                    curation_note=''))


    

In [ ]:
imaging.Activity()

In [ ]:
imaging.MotionCorrection.heading

In [ ]:
imaging.MotionCorrection.populate(**populate_settings)
imaging.Segmentation.populate(**populate_settings)
imaging.MaskClassification.populate(**populate_settings)
imaging.Fluorescence.populate(**populate_settings)
imaging.Activity.populate(**populate_settings)

# Testing if data is loaded correctly

In [ ]:
session_key = (imaging.Fluorescence & 'subject = "jm003"').fetch('KEY')[0]
print(session_key)

In [ ]:
imaging.MotionCorrection & session_key

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
avg_im

In [ ]:
avg_im = (imaging.MotionCorrection.Summary & session_key).fetch('average_image')
plt.imshow(np.array(avg_im[0]))
plt.show()
plt.imshow(np.array(avg_im[1]))
plt.show()
plt.imshow(np.array(avg_im[2]))
plt.show()

In [ ]:
from scipy.stats import zscore

In [ ]:
imaging.Activity.Trace & session_key

In [ ]:
F = (imaging.Fluorescence.Trace & session_key).fetch('fluorescence')

In [ ]:
F_toplot = zscore(np.stack(F), 1)

In [ ]:
plt.figure(figsize=(20,20), dpi=300)
plt.imshow(F_toplot,cmap='Greys', vmin=0, vmax=3.65)